# UDP

<img src='res/udp02.png' width=500px />

<img src='res/udp01.png' width=500px />

1. Hash table to store udp socks. key is the port `number & (UDP_HTABLE_SIZE - 1)`

### static struct sock *udp_v4_lookup_longway

```c
/* UDP is nearly always wildcards out the wazoo, it makes no sense to try
 * harder than this. -DaveM
 */
static struct sock *udp_v4_lookup_longway(u32 saddr, u16 sport,
					  u32 daddr, u16 dport, int dif)
{
	struct sock *sk, *result = NULL;
	struct hlist_node *node;
	unsigned short hnum = ntohs(dport);
	int badness = -1;

	sk_for_each(sk, node, &udp_hash[hnum & (UDP_HTABLE_SIZE - 1)]) {
		struct inet_sock *inet = inet_sk(sk);

		if (inet->num == hnum && !ipv6_only_sock(sk)) {
			int score = (sk->sk_family == PF_INET ? 1 : 0);
			if (inet->rcv_saddr) {
				if (inet->rcv_saddr != daddr)
					continue;
				score+=2;
			}
			if (inet->daddr) {
				if (inet->daddr != saddr)
					continue;
				score+=2;
			}
			if (inet->dport) {
				if (inet->dport != sport)
					continue;
				score+=2;
			}
			if (sk->sk_bound_dev_if) {
				if (sk->sk_bound_dev_if != dif)
					continue;
				score+=2;
			}
			/* xitongsys
			score == 9，所有条件都满足，直接返回
			*/
			if(score == 9) { 
				result = sk;
				break;
			} else if(score > badness) { //xitongsys： 找到匹配最多（score最大）的sock
				result = sk;
				badness = score;
			}
		}
	}
	return result;
}
```

1. 从hash表中查找匹配的sock。因为一个port可以绑定多个sock对象，这里采用了最有匹配。